In [1]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=localhost port=5432 dbname=cloud_user user=cloud_user password=cloud_user"

In [5]:
create_table_query = '''CREATE TABLE tips (
    ID SERIAL PRIMARY KEY,
    weekday varchar (10),
    meal_type varchar (10),
    wait_staff varchar (10),
    party_size smallint,
    meal_total float4,
    tip float4
); '''

try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()

    # Send sql query to request
    cur.execute(create_table_query)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')

PostgreSQL connection is closed
Records:
 None


In [7]:
try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()

    with open('./tips.csv', 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'tips', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("tips table populated")

PostgreSQL connection is closed
tips table populated


In [8]:
def db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cur.fetchall()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

In [9]:
select_query = '''SELECT * FROM tips LIMIT 5;'''

records = db_server_fetch(select_query)
print(records)

PostgreSQL connection is closed
[(1, 'Saturday', 'Dinner', 'Marcia', 2, 100.64, 16.23), (2, 'Friday', 'Dinner', 'Marcia', 2, 109.84, 5.99), (3, 'Friday', 'Lunch', 'Jan', 4, 90.5, 22.04), (4, 'Monday', 'Dinner', 'Marcia', 1, 60.01, 8.77), (5, 'Monday', 'Breakfast', 'Jan', 1, 10.88, 1.68)]


In [10]:
def db_server_change(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cxn.commit()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

In [11]:
add_data = '''INSERT INTO tips
    (id, weekday, meal_type, wait_staff, party_size, meal_total, tip)
    VALUES
    (504, 'Saturday', 'Breakfast', 'Alfred', 1, 10.76, 0.50);'''

db_server_change(add_data)

PostgreSQL connection is closed


In [12]:
select_query = '''SELECT * FROM tips WHERE wait_staff='Alfred';'''

records = db_server_fetch(select_query)
print(records)

PostgreSQL connection is closed
[(504, 'Saturday', 'Breakfast', 'Alfred', 1, 10.76, 0.5)]


In [13]:
def pandas_db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Send sql query to request and create dataframe
        df = pd.read_sql(sql_query, cxn)

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cxn.close()
            print("PostgreSQL connection is closed")
        return df

In [14]:
select_query = '''SELECT * FROM tips WHERE wait_staff='Alfred';'''

alfred_df = pandas_db_server_fetch(select_query)
alfred_df.head()

PostgreSQL connection is closed


,id,weekday,meal_type,wait_staff,party_size,meal_total,tip
0,504,Saturday,Breakfast,Alfred,1,10.76,0.5


In [15]:
tips_df = pandas_db_server_fetch('''SELECT * FROM tips;''')

PostgreSQL connection is closed


In [16]:
tips_df.head()

,id,weekday,meal_type,wait_staff,party_size,meal_total,tip
0,1,Saturday,Dinner,Marcia,2,100.64,16.23
1,2,Friday,Dinner,Marcia,2,109.84,5.99
2,3,Friday,Lunch,Jan,4,90.50,22.04
3,4,Monday,Dinner,Marcia,1,60.01,8.77
4,5,Monday,Breakfast,Jan,1,10.88,1.68
